https://huggingface.co/docs/transformers/main_classes/output

https://www.kaggle.com/code/debarshichanda/bert-multi-label-text-classification/notebook

In [1]:
import os
import re
import string
import json
#import emoji
import numpy as np
import pandas as pd
from sklearn import metrics
from bs4 import BeautifulSoup
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.max_columns", None)

In [2]:
df_train = pd.read_csv("train.csv")
df_dev   = pd.read_csv("val.csv")
#df_train = pd.read_csv("train.csv", header=None, names=['Text', 'Class', 'ID'])
#df_dev = pd.read_csv("val.csv", sep='\t', header=None, names=['Text', 'Class', 'ID'])
df_train, df_dev

(                                                    Text       ID  anger  \
 0                         WHY THE FUCK IS BAYLESS ISOING  eezlygj      1   
 1                            To make her feel threatened  ed7ypvh      0   
 2                                 Dirty Southern Wankers  ed0bdzj      1   
 3      OmG pEyToN iSn'T gOoD eNoUgH tO hElP uS iN tHe...  edvnz26      0   
 4      Yes I heard abt the f bombs! That has to be wh...  ee3b6wu      0   
 ...                                                  ...      ...    ...   
 28422  Added you mate well I’ve just got the bow and ...  edsb738      0   
 28423  Always thought that was funny but is it a refe...  ee7fdou      0   
 28424  What are you talking about? Anything bad that ...  efgbhks      1   
 28425            More like a baptism, with sexy results!  ed1naf8      0   
 28426                                    Enjoy the ride!  eecwmbq      0   
 
        fear  joy  sadness  surprise  
 0         0    0        0         

In [3]:
#def list_of_classes(row):
#    classes = ["anger", "fear", "joy", "sadness", "surprise"]
#    arr = [1 if emotion=="1" else 0 for emotion in classes]
#    print(arr)
#    return arr
#df_train = df_train.apply(list_of_classes, axis=1, result_type='expand')
#df_train['Len of classes'] = df_train['List of classes'].apply(lambda x: len(x))
#df_dev['List of classes'] = df_dev['Class'].apply(lambda x: x.split(','))
#df_dev['Len of classes'] = df_dev['List of classes'].apply(lambda x: len(x))
#df_train.head(10)

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [5]:
MAX_LEN = 200
TRAIN_BATCH_SIZE = 64
VALID_BATCH_SIZE = 64
EPOCHS = 10
LEARNING_RATE = 2e-5
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
target_cols = [col for col in df_train.columns if col not in ['Text', 'ID']]
target_cols

['anger', 'fear', 'joy', 'sadness', 'surprise']

In [6]:
class BERTDataset(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.df = df
        self.max_len = max_len
        self.text = df.Text
        self.tokenizer = tokenizer
        self.targets = df[target_cols].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [7]:
train_dataset = BERTDataset(df_train, tokenizer, MAX_LEN)
valid_dataset = BERTDataset(df_dev, tokenizer, MAX_LEN)

In [8]:
train_loader = DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4, shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=VALID_BATCH_SIZE, num_workers=4, shuffle=False, pin_memory=True)

In [9]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 

class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.roberta = AutoModel.from_pretrained('roberta-base')
#         self.l2 = torch.nn.Dropout(0.3)
        self.fc = torch.nn.Linear(768,5)
    
    def forward(self, ids, mask, token_type_ids):
        _, features = self.roberta(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=False)
#         output_2 = self.l2(output_1)
        output = self.fc(features)
        return output

model = BERTClass()
model.to(device);

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = AdamW(params=model.parameters(), lr=LEARNING_RATE, weight_decay=1e-6)

In [12]:
enumerate(train_loader, 0)

KeyboardInterrupt: 

In [11]:
i_break = 2
def train(epoch):
    model.train()
    for i, data in enumerate(train_loader, 0):
        print(f"Epoch {epoch}, step {i}")
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs = model(ids, mask, token_type_ids)
        loss = loss_fn(outputs, targets)
        if i%i_break==0:
            print(f'Epoch: {epoch}, Loss: {loss.item()}')
            break
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
for epoch in range(EPOCHS):
    train(epoch)

KeyboardInterrupt: 

In [ ]:
def validation():
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(valid_loader, 0):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets